<a href="https://colab.research.google.com/github/rodrigossilveira/DS_projects/blob/main/Stocks_independence_by_itemset_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stock independence by itemset mining
 When investing a very important rule is  "Don't put all your eggs in the same basket", meaning one should diversify it's investments so the performance of some can offset the negative performance of others and protect the value of the portfolio. But diversification is not a trivial taks as many stocks tend to move together. Here this problem will be tackled trying to use itemset mining as a way to discovering wich stocks are connected/dependent. 

In [ ]:
import pandas as pd
display("Ready!")

'Ready!'

We will be using here historical data from B3, the company responsible for stock exchange here in Brazil. The data is available at [their website](https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/cotacoes-historicas/) where there is also a [guide](https://drive.google.com/file/d/1gMkCG4HvlWokYN7c_NCGvSnULRhNzQNA/view?usp=sharing) to understanding the data layout. The dataset has three parts: The first line is the header and the last the trailer, both with some information about the year of the data, and it's source. The rest of the dataset is fixed width formatted, meaning that each column has a fixed interval in the dataset, with diferent sizes. The guide specifies the starting and ending character of each columns, as well as their names, and we are going to use it to get the data right. 

In [28]:
#The starting points of each column and the ending point of the last one were written bellow
starting_points = [1,3,11,13,25,28,40,50,53,57,70,83,96,109,122,135,148,153,171,189,202,203,211,218,231,243,245]
width = []
#By subtracting each with the previous one we will obtain the columns intervals
for i in range(len(starting_points)-1):
  width.append(starting_points[i+1] - starting_points[i])
#That we can finally use to import the data
path = '/content/drive/MyDrive/Colab_Data/Stocks Independence/COTAHIST_A2020.TXT'
stocks_results = pd.read_fwf(path, widths=width, header = 0)
#The first and last line that read 99COTAHIST.2020BOVESPA 2020123000001251648
#were not considered in the dataframe
stocks_results.head(-5)

,00,COTAHIST,.2,020BOVESPA 2,020,1230,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1,20200102,2.0,AALR3,10,ALLIAR,ON NM,NaN,R$,1829.0,1900.0,1828.0,1868.0,1900.0,1899.0,1901.0,2443.0,585800.0,1.094620e+09,0.0,0.0,99991231.0,1.0,0.0,BRAALRACNOR6,10.0
1,1,20200102,2.0,AAPL34,10,APPLE,DRN,NaN,R$,12000.0,12134.0,12000.0,12073.0,12134.0,10550.0,12134.0,9.0,12700.0,1.533380e+08,0.0,0.0,99991231.0,1.0,0.0,BRAAPLBDR004,13.0
2,1,20200102,2.0,ABCB4,10,ABC BRASIL,PN EJ N2,NaN,R$,2000.0,2030.0,1982.0,2005.0,2030.0,2008.0,2030.0,3979.0,870400.0,1.745788e+09,0.0,0.0,99991231.0,1.0,0.0,BRABCBACNPR4,13.0
3,1,20200102,2.0,ABEV3,10,AMBEV S/A,ON EJ,NaN,R$,1886.0,1925.0,1878.0,1912.0,1920.0,1919.0,1920.0,31581.0,16011300.0,3.062348e+10,0.0,0.0,99991231.0,1.0,0.0,BRABEVACNOR1,12.0
4,1,20200102,2.0,ADHM3,10,ADVANCED-DH,ON,NaN,R$,255.0,255.0,235.0,244.0,248.0,248.0,249.0,307.0,193400.0,4.719270e+07,0.0,0.0,99991231.0,1.0,0.0,BRADHMACNOR9,10.0


In [31]:
#Here we add the column names according to the data guide
column_names = ['Registry type', 'Session date','BDI code','Stock Symbol',
               'Type of Market',  'Stock Issuing Company Name', 
                'Stock Specification', 'Prazo',  'Currency used',
                'Openning price', 'Maximum price', 'Minimal Price','Mean price',
                'Last exchange price', 'Best buying offer', 'Best selling offer',
                'Number of transactions', 'Number of stocks exchanged',
                'Volume of stocks exchanged', 'Price for options market', 
                'Price correction indicator', 'Due date', 'Quotation Factor',
                'Options price in points', 'Internal stock code',
                'Stock Distribuiton number']
stocks_results.columns = column_names

#Some columns came without the decimal separator, so we need to divide them by 100
decimal_columns = ['Openning price', 'Maximum price', 'Minimal Price',
                   'Mean price','Last exchange price', 'Best buying offer',
                   'Best selling offer','Volume of stocks exchanged', 
                   'Price for options market']
stocks_results.head(-5)                   

,Registry type,Session date,BDI code,Stock Symbol,Type of Market,Stock Issuing Company Name,Stock Specification,Prazo,Currency used,Openning price,Maximum price,Minimal Price,Mean price,Last exchange price,Best buying offer,Best selling offer,Number of transactions,Number of stocks exchanged,Volume of stocks exchanged,Price for options market,Price correction indicator,Due date,Quotation Factor,Options price in points,Internal stock code,Stock Distribuiton number
0,1,20200102,2.0,AALR3,10,ALLIAR,ON NM,NaN,R$,1829.0,1900.0,1828.0,1868.0,1900.0,1899.0,1901.0,2443.0,585800.0,1.094620e+09,0.0,0.0,99991231.0,1.0,0.0,BRAALRACNOR6,10.0
1,1,20200102,2.0,AAPL34,10,APPLE,DRN,NaN,R$,12000.0,12134.0,12000.0,12073.0,12134.0,10550.0,12134.0,9.0,12700.0,1.533380e+08,0.0,0.0,99991231.0,1.0,0.0,BRAAPLBDR004,13.0
2,1,20200102,2.0,ABCB4,10,ABC BRASIL,PN EJ N2,NaN,R$,2000.0,2030.0,1982.0,2005.0,2030.0,2008.0,2030.0,3979.0,870400.0,1.745788e+09,0.0,0.0,99991231.0,1.0,0.0,BRABCBACNPR4,13.0
3,1,20200102,2.0,ABEV3,10,AMBEV S/A,ON EJ,NaN,R$,1886.0,1925.0,1878.0,1912.0,1920.0,1919.0,1920.0,31581.0,16011300.0,3.062348e+10,0.0,0.0,99991231.0,1.0,0.0,BRABEVACNOR1,12.0
4,1,20200102,2.0,ADHM3,10,ADVANCED-DH,ON,NaN,R$,255.0,255.0,235.0,244.0,248.0,248.0,249.0,307.0,193400.0,4.719270e+07,0.0,0.0,99991231.0,1.0,0.0,BRADHMACNOR9,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251637,1,20201230,82.0,YDUQM290,80,YDUQE,ON NM,0.0,R$,18.0,18.0,18.0,18.0,18.0,5.0,33.0,1.0,100.0,1.800000e+03,2900.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0
1251638,1,20201230,82.0,YDUQM310,80,YDUQE FM,ON NM,0.0,R$,49.0,49.0,49.0,49.0,49.0,47.0,0.0,2.0,2000.0,9.800000e+04,3100.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0
1251639,1,20201230,82.0,YDUQM320,80,YDUQE FM,ON NM,0.0,R$,90.0,90.0,75.0,81.0,79.0,78.0,0.0,11.0,2200.0,1.786000e+05,3200.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0
1251640,1,20201230,82.0,YDUQM330,80,YDUQE FM,ON NM,0.0,R$,115.0,127.0,115.0,116.0,127.0,0.0,150.0,6.0,8500.0,9.895000e+05,3300.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0


In [29]:
stocks_results.head(-5)

,00,COTAHIST,.2,020BOVESPA 2,020,1230,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1,20200102,2.0,AALR3,10,ALLIAR,ON NM,NaN,R$,1829.0,1900.0,1828.0,1868.0,1900.0,1899.0,1901.0,2443.0,585800.0,1.094620e+09,0.0,0.0,99991231.0,1.0,0.0,BRAALRACNOR6,10.0
1,1,20200102,2.0,AAPL34,10,APPLE,DRN,NaN,R$,12000.0,12134.0,12000.0,12073.0,12134.0,10550.0,12134.0,9.0,12700.0,1.533380e+08,0.0,0.0,99991231.0,1.0,0.0,BRAAPLBDR004,13.0
2,1,20200102,2.0,ABCB4,10,ABC BRASIL,PN EJ N2,NaN,R$,2000.0,2030.0,1982.0,2005.0,2030.0,2008.0,2030.0,3979.0,870400.0,1.745788e+09,0.0,0.0,99991231.0,1.0,0.0,BRABCBACNPR4,13.0
3,1,20200102,2.0,ABEV3,10,AMBEV S/A,ON EJ,NaN,R$,1886.0,1925.0,1878.0,1912.0,1920.0,1919.0,1920.0,31581.0,16011300.0,3.062348e+10,0.0,0.0,99991231.0,1.0,0.0,BRABEVACNOR1,12.0
4,1,20200102,2.0,ADHM3,10,ADVANCED-DH,ON,NaN,R$,255.0,255.0,235.0,244.0,248.0,248.0,249.0,307.0,193400.0,4.719270e+07,0.0,0.0,99991231.0,1.0,0.0,BRADHMACNOR9,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251637,1,20201230,82.0,YDUQM290,80,YDUQE,ON NM,0.0,R$,18.0,18.0,18.0,18.0,18.0,5.0,33.0,1.0,100.0,1.800000e+03,2900.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0
1251638,1,20201230,82.0,YDUQM310,80,YDUQE FM,ON NM,0.0,R$,49.0,49.0,49.0,49.0,49.0,47.0,0.0,2.0,2000.0,9.800000e+04,3100.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0
1251639,1,20201230,82.0,YDUQM320,80,YDUQE FM,ON NM,0.0,R$,90.0,90.0,75.0,81.0,79.0,78.0,0.0,11.0,2200.0,1.786000e+05,3200.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0
1251640,1,20201230,82.0,YDUQM330,80,YDUQE FM,ON NM,0.0,R$,115.0,127.0,115.0,116.0,127.0,0.0,150.0,6.0,8500.0,9.895000e+05,3300.0,0.0,20210118.0,1.0,0.0,BRYDUQACNOR3,10.0
